*Материал оформила и подготовила* Виктория Фирсанова

# **Обучение LSTM для генерации текста**

# Импорт классов и функций

In [ ]:
import numpy # NumPy-массивы будут использоваться для работы с обучающей выборкой

# импортируем из библиотеки Keras функции для построения нейросети

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM 
from tensorflow.keras.callbacks import ModelCheckpoint #эта функция позволит сохранить вычисленные сетью веса
from keras.utils import np_utils


# Загрузим данные для обучения

Все загруженные или полученные в результате работы программы файлы в *Google.Colab* можно увидеть (а также скачать или удалить) во вкладе *Files* слева.


In [ ]:
from google.colab import files # функция для загрузки любых файлов
files.upload()

In [ ]:
!ls # проверка: отображает список имен загруженных файлов

# Загрузим текст и посмотрим на наши обучающие данные

Предварительно наши данные придется немного обработать.

In [ ]:
filename = "output.txt" # имя файла с обучающей выборкой
raw_text = open(filename).read()

# Все символы можно привести к нижнему регистру.
# Это уменьшит словарный запас, который наша сеть должна будет выучить.

raw_text = raw_text.lower() 

# В ходе экспериментов я выяснила, что моя выборка слишком объемная для этой задачи, 
# поэтому мне понадобилось ее "обрезать".
# Мне также пришлось избавиться от всех символов кроме букв и цифр.
# Их наличие влияет на длину словаря - сеть будет обучаться с большим трудом, если словарь окажется слишком длинным.
# В моем случае с интернет-текстами - это особенно важно. 
# Кроме обычных знаков препинания, в выборку могут попасть и другие символы, например, эмодзи.

# Подготовка данных для обучения

Мы не можем построить модель из символов (*букв и знаков препинания*) напрямую. Но мы можем преобразовать символы в числа.

Для обучения нейросети мы будем использовать посимвольную модель.

In [ ]:
# Преобразуем уникальные символы в целые числа

chars = sorted(list(set(text))) # используем метод set()

# метод конвертирует любой итерируемый объект (iterable) в отдельный элемент
# получаем список символов, сортированный в алфавитном порядке

print("Взглянем на получившийся список символов:", chars)

In [ ]:
# Получим словарь символов и присвоим каждому элементу этого словаря индекс.
# На предыдущем этапе мы отсортировали элементы словаря в алфавитном порядке.
# Теперь порядковый номер каждого элемента в этом списке "кодирует" соответствующий символ.

char_to_int = dict((c, i) for i, c in enumerate(chars)) 

n_chars = len(text)
n_vocab = len(chars)

print("Количество символов в базе данных для обучения: ", n_chars)
print("Длина словаря символов: ", n_vocab)
print("Словарь:", char_to_int)

# Подготовим выборку 
- Разделим базу данных на последовательности текстов заданной длины. 
- Преобразуем символы в целые числа, используя созданный на предыдущем этапе частотный словарь.

In [ ]:
seq_length = 100 # задаем длину отрывков для обучения
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1): 

	seq_in = text[i:i + seq_length]
	seq_out = text[i + seq_length]

	dataX.append([char_to_int[char] for char in seq_in]) 
	dataY.append(char_to_int[seq_out])
 
n_patterns = len(dataX) 

print("Всего шаблонов для обучения: ", n_patterns)

# Тренировочные данные подготовлены!
Теперь нужно преобразовать их так, чтобы они подходили для работы с Keras.

In [ ]:
#Преобразуем список входных последовательностей к виду [samples (образцы), time steps (временные шаги), features(особенности)].
#Это вид, подходящий для обучения сети LSTM.

X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

#Нормализация

#Нам нужно изменить масштаб целых чисел так, чтобы LSTM могла "исследовать" данные.
#Поскольку по умолчанию мы используем функцию активации сигмоид: 0 <= n <= 1

#В ходе преобразований нам нужно получить "одну горячую кодировку" (one hot encoding).
#Это более простое представление: разреженный вектор длиной n_vocab, полный нулей.
#Вектор содержит одну единицу, которая представляет собой искомый символ.

X = X / float(n_vocab)

#Приводим наши паттерны к виду "one hot encoding" (пример: если n=3, то выглядеть это будет так: [0. 0. 1. 0.])
#Cейчас они представлены в виде чисел, каждое из которых представляет собой индекс частотности данного символа.

y = np_utils.to_categorical(dataY)

# Обучаем LSTM-модель

Параметры модели можно (и нужно, если мы хотим получить лучший результат) изменять. 

Мы сохраняем веса модели после каждой эпохи обучения, можно смело переобучать нейросеть и экспериментировать с параметрами! 

*Все наши веса сохраняться во вкладке "Files": лучшие следует сохранить, а худшие - удалить.*

**Как долго обучается модель?**

Все завивисит от количества эпох, длины словаря и объема обучающей выборки. Эту модель можно назвать неторопливой. Она может обучаться и 2 часа, и 12. Но через 12 часов карета превратится в тыкву, и сессия в Google.Colab прервется.

**Как определить лучший вариант?**

Из всех вариантов, полученных в ходе обучения, лучшим будет тот, в котором меньше всего потерь => мы ищем минимальное значение параметра *loss*.

В следующем примере *loss* первой эпохи равен 3.1670, а второй - 3.0254. Соответственно, нам нужны веса, который сохарнились в файл с названием *weights-improvement-02-3.0254.hdf5*.

```
Epoch 1/2
2343/2343 [==============================] - ETA: 0s - loss: 3.1670
Epoch 00001: loss improved from inf to 3.16704, saving model to weights-improvement-01-3.1670.hdf5
2343/2343 [==============================] - 1454s 621ms/step - loss: 3.1670

Epoch 2/2
2343/2343 [==============================] - ETA: 0s - loss: 3.0254
Epoch 00002: loss improved from 3.16704 to 3.02536, saving model to weights-improvement-02-3.0254.hdf5
2343/2343 [==============================] - 1456s 621ms/step - loss: 3.0254
```

**За счет каких параметров я могу улучшить качество работы модели?**

Вот несколько идей, которые сработали с моими данными:
- увеличить количество эпох (10 -> 20 -> 50 -> 100);
- уменьшить batch size (128 -> 64);
- добавить еще один слой: 
```
model.add(LSTM(256)) 
```

**Как увеличить скорость обучения?**

Вот что мне помогло:

- я удалила символы из обучающей выборки
- и уменьшила ее объем

**НО!** из-за малого объема выборки, НС будет скорее воспроизводить тексты из обучающей выборки, чем генерировать оригинальные.

*Цель:* составить не очень большую, но емкую выборку, на основе которой программа могла бы генерировать оригинальные тексты.



In [ ]:
# Задаем модель

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Задаем параметры для сохранения весов модели по ходу обучения

filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

# Генерация текста с помощью LSTM-модели

- Загружаем веса, которые вычислила нейросеть в ходе обучения
- Компилируем модель
- Пишем функцию для генерации текста
- Наблюдаем за результатом

In [ ]:
files.upload() # при необходимости загружаем предобученную модель

In [ ]:
# загружаем файл весов с наименьшим значением потерь

filename = "weights-improvement-40-1.2724.hdf5"
model.load_weights(filename) 
model.compile(loss='categorical_crossentropy', optimizer='adam') # и компилируем модель

In [ ]:
#Чтобы сопоставить символы с целыми числами,
#создаем "обратное отображение" (char to integer -> integer to char).

#Это нужно для того, чтобы мы, homo sapiens, могли понять предсказания искусственного интеллекта :)

int_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
# Генерация текста = прогнозы нашей модели

# Задается случайная последовательность чисел.
# Она используется для генерации следующего символ.
# Последовательность чисел обновляется, сгенерированный символ добавляется в конец, первый - удаляется. 

# Процесс повторяется до выполнения заданного условия.
# Например, мы можем задать условие, при котором последовательность не должна превышать 1000 символов.

# Мы можем выбрать случайный шаблон в качестве источника для генерации последовательности символов.

import sys

# Выбираем из обучающей выборки случайный источник для генерации текста с помощью генератора случайных чисел.

start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# Генерируем текст!

for i in range(5000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nEnd.")

**Список источников:**

1. Hochreiter, S. (1997). "Long Short-term Memory". Neural Computation 9(8): 1735-80. DOI: 10.1162/neco.1997.9.8.1735
2. Brownlee, J. (2016). Text Generation With LSTM Recurrent Neural Networks in Python with Keras. Machine Learning Mastery. URL: https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/ 
